# Implémentation d'un système de tweet en Django

## Prérequis

### Installation de Django

Pour installer Django, dans GitBash:
*python -m pip install Django*

### Installation de PostGreSQL

Django utilise par défaut le système de bases de données SQLite, installé dans Python. Cependant le nouveau portail des élèves en développement utilise PostGreSQL, nous l'utiliserons donc pour nous familiariser.

Rendez vous simplement sur https://www.postgresql.org/download/ pour installer PSQL. Il faut ensuite installer la librairie psycopg2 via *pip install psycopg2*. Elle permet d'utiliser une base SQL dans un projet Django.

Vérifiez que l'installation s'est bien effectuée en tapant la commande *psql* dans un terminal. Celle-ci ouvre une console PostgreSQL dans laquelle vous pouvez interagir avec les différentes bases du système (pour en sortir, tapez *\q*).

Il y a plein de façons de créer une base de données dans PostgreSQL. Choisissons ici de passer par le Shell SQL (facilement trouvable dans vos programmes). Il faut lancer le Shell, puis taper sur Entrée 5 fois. Nous sommes alors sur le compte par défaut nommé **postgres**. La commande à taper pour créer une database est: *CREATE DATABASE nom_de_la_base;*. Notre base de données s'appelera **tweetbase**.

La commande *\l* permet ensuite de lister les bases de données crées. Verifiez que la votre l'est bien, vous devriez obtenir ceci:

![database](images_presa/database.jpg)

## Lancement du Projet

Pour lancer un prjet django, lancer la commande:

*django-admin startproject <project_name>*

Choisissons **backend** pour nom de projet.

Voyons ce que *startproject* a crée :

    backend/

        manage.py

        backend/

            __init__.py

            settings.py

            urls.py

            asgi.py
        
            wsgi.py


Faisons un petit inventaire des fichiers crées :

* *manage.py* est un exectuable python en ligne de commande qui nous permettra d'interagir avec le projet Django. Nous verrons par la suite quelques exemples de son utilisation.

* Le sous répertoire *backend* effectif du projet. Quasiment tout ce que nous coderons et importerons se trouvera dans ce fichier.
* *backend/\__init__.py* est un fichier vide qui indique à Python que ce répertoire doit être considéré comme un paquet. Nous ne nous en préoccuperons pas à notre niveau.

* *backend/settings.py* est le fichier qui gère les réglages et la configuration de notre projet Django. Nous y changerons des paramètres dans la suite.

* *backend/urls.py* servira à déclarer les URL utilisées par notre projet Django.

* *asgi.py* est un fichier qui sert de points d'entrée pour les serveurs Web compatibles aSGI pour déployer notre projet. Nous n'irons pas jusque là.

* *wsgi.py* même chose mais pour les serveurs Web compatibles WSGI.

## Lancement du serveur de développement

Afin de tester notre projet en phase de développement, il suffit de lancer la commande : *python manage.py runserver*. Le serveur est alors lancé en local sur l'adresse *localhost:8000* (le port est changeable, demandez nous si besoin).

Si vous essayez de vous y connecter, vous obtiendrez peut-être le message suivant :

![erreur_lancement](images_presa/erreur_lancement.jpg)

Ne paniquez pas, restez calme. Il faut simplement ajouter l'adresse *localhost* à notre fichier settings, à l'endroit suivant :

![settings_url](images_presa/settings_url.jpg)

Cette ligne de code signale simplement à Django qu'il est autorisé à lancer le serveur sur l'adresse indiquée. Vous n'avez pas besoin d'interrompre le programme *manage.py runserver* pendant vos modifications, elles sont automatiquement prises en compte. Si vous actualisez maintenant la page, vous devriez voir apparaître une jolie fusée vous signifiant que le lancement du projet est réussi !

## Création de l'application *app*

Maintenant que notre environnement de travail est en place, il est temps de créer une application. Une application est une application Web qui fait quelque chose – par exemple un système de blog, une base de données publique ou une petite application de sondage. Le projet est quant à lui un ensemble d'applications qui constituent un site Web. 

Pour créer notre applications, plaçons nous dans le même répertoire que *manage.py* et lançons la commande : *python manage.py startapp app*

Cela a pour effet de créer un répertoire *tweet*, structuré de la façon suivante :

    app/

        __init__.py

        admin.py

        apps.py

        migrations/

            __init__.py

        models.py

        tests.py
        
        views.py

Là encore, un petit inventaire des fichiers présents :

* *\__init__.py* déclare que ce répertoire est une librairie

* *admin.py* : configuration de l'espace d'administration

* *apps.py* : configuration de l'application

* *migrations* : fichiers qui modifient la structure de la base de données

* *models.py* : modèles de l'application (nous y reviendrons)

* *tests.py* : contiendra les tests de notre application

* *views.py* : vues de l'application (nous y reviendrons)

Chaque création d'application nécessite de déclarer celle-ci à notre projet, c'est à dire dans *settings.py* comme ceci :

![appconfig](images_presa/appconfig.jpg)

## Déclarer une base de données

Pour informer Django qu'il doit utiliser la database **tweetbase**, il faut modifier *settings.py* comme suit:

![settings database](images_presa/settings_database.jpg)

Nous déclarons d'abord à Django que la base de données est gérée par PostgreSQL, puis son nom, le nom d'utilisateur, le mot de passe correspondant (faite attention à bien mettre le même mot de passe que celui donné lors de l'installation de PostgreSQL) et le port à utiliser.

Parlons maintenant un peu des migrations Django. Elles sont la manière par laquelle Django propage les changements que nous effectuons sur le code dans les bases de données. Ces migrations se font quasiment automatiquement, il faut seulement connaître quelques commandes qui permettent de les déclencher. Nous aurons besoin de deux d'entre elles:

* *migrate* qui est responsable de l’exécution et de l’annulation des migrations.
* *makemigrations* qui est responsable de la création de nouvelles migrations en fonction des modifications que vous avez apportées aux modèles.

Comme nous venons de créer notre base de données, nous devons effectuer des modifications dans la base de données nécessaires pour faire tourner le projet. Il faut lancer la commande *python manage.py migrate*. Et voilà ! Notre projet est maintenant lié à notre base de données SQL, nous verrons comment nous en servir plus tard.

Nous pouvons maintenant développer notre application de gestion de tweets. Pour cela, il est nécessaire de connaître le principe de l'architecture MVT (Model-View-Template) et le fonctionnement d'une base de données (très sommairement !).

# L'architecture MVT

## Le Modèle

Le modèle interagit avec la base de données. Sa mission est d'aller chercher dans une base de données les items correspondant à une requête et de renvoyer une réponse exploitable par le programme.

Les modèles Django s'appuient sur un ORM (Object Relationnal Mapping), ce qui nous évite nombre de problèmes et traduit directement les résultats d'une requête SQL en objet Python.

Nous verrons leur fonctionnement dans la suite, lorsque nous parlerons de Django REST Framework et créerons nos premiers modèles.

## Le Template

Les templates sont les fichiers HTML qui seront la vitrine de notre site et qui pourront recevoir des informations telles que des objets Python, envoyés par des vues. Nous les placerons dans le dossier *app/templates* que nous nous empressons de créer. Rappelons l'architecture de notre application :


    app/

        __init__.py

        admin.py

        apps.py

        migrations/

            __init__.py

        models.py

        tests.py
        
        views.py

        templates/

Nous n'aurons pas besoin d'utliser des templates pour notre application de tweets, puisque le frontend sera géré par React. Mais si cela vous intéresse, leur fonctionnement ressemble à celui de Jinja2. Nous avons crée pour notre projet un template de test, noté *index.html* que vous pouvez consulter via notre git. Il sert simplement à envoyer une requête (via un form) qui correspondrait à celle qu'enverrait le frontend React au backend Django.

## La Vue

La vue est l'élément le plus important de la structure. Il est chargé de recevoir les requêtes HTTP du navigateur et de construire la réponse à lui renvoyer.

Très concrètement, la vue reçoit une requête HTTP, puis elle la traite en faisant notamment appel à des modèles. Elle renvoie la réponse sous forme d'un template complété par les données fournies par le programme contenu dans la vue.

Toutes les vues de notre application *tweet* seront regroupées dans *views.py*, chaque vue étant associée à une URL.

La vue de base est la vue *index* qui, par convention, est celle qui traite la page d'accueil de notre application, donc l'URL *http://127.0.0.1/<nom_appli>*.

Prenons par exemple la vue index du site **Minception**

In [3]:
from django.shortcuts import render

def index(request):
    return render(request, 'accueil/index.html')

Cette vue est sans doute la plus simple qu'il soit possible de faire. Lorsqu'on se connecte à la page d'accueil du site, elle renvoie simplement le template *index.html*. Il est évidemment possible de faire des vues bien plus complexes, nous le verrons dans la suite.

Notons tout de même la présence d'un import : celui de la fonction *render* du module *django.shortcuts*. Cette fonction convertit le template (et les données fournies sous forme d'un dictionnaire si il y en a) en réponse HTTP interprétable par le navigateur. Elle est très puissant puisqu'elle permet d'intégrer toutes sortes de données à notre template via un simple dictionnaire Python.

Ajoutez donc index.html (voir dans le dossier app/templates de notre Git) à votre dossier app/templates, puis le code suivant dans views.py :

In [ ]:
from django.shortcuts import render

def index(request):
    return render(request, 'index.html')

Vous vous demandez sans doute comment Django sait qu'il faut lancer cette vue lors de la connexion à la page d'accueil. Cela fait appel au fichier urls.py, et nous allons le voir tout de suite !

## Ajout d'une route

La première étape est d'inclure les routes de l'application *app* dans notre projet *backend*. Nous aurons en effet plusieurs fichiers *urls.py*. Le premier étant celui se trouvant dans *backend*, et les autres chacun dans leur applications respectives. Dans notre cas, nous aurons donc un fichier *backend/urls.py* et un fichier *app/urls.py*. Cela permet plus de clarté dans la construction globale d'un projet Django. Il faut cependant préciser dans à Django qu'il doit utiliser les routes que nous indiquerons dans *app/urls.py*. Pour cela nous écrivons dans *backend/urls.py* :

![urls_project](images_presa/urls_project.jpg)

Faites notamment attention ici aux imports ! Ceux effectués de base par Django ne suffisent pas. Nous avons ajouté ceux de *url* et *include*.

Créeons ensuite le fichier *app/urls.py* et écrivons y le code qui suit :

![urls](images_presa/urls.jpg)

Nous importons d'abord la méthode *url* du module *django.conf.urls*, et les vues de notre application.
La variable *urlpatterns* est une liste comportant tous les URLs accessibles sur notre site. Nous en avons indiqué une seule pour l'instant, celle de la page d'accueil. La méthode *url()* associe donc à un URL une des vues de notre choix. Nous avons donc associé ici la vue *index* à l'URL http://127.0.0.1/

Voici la page d'accueil de test que nous avons crée.

![index](images_presa/index.jpg)

Il sera intéressant pour vous dde consulter le code html derrière cette page pour se faire un idée de la requête envoyée lorsque le form est complété. Nous reparlerons de cette requête dans la suite.

Vous noterez par ailleurs la présence de la ligne {% csrf_token %} dans le code html. Elle permet de se protéger contre les attaques de type **Cross Site Request Forgery**, mais nous ne nous y attarderons pas.

Et le tour est joué ! Vous pouvez donc choisir de définir vos URLs à partir du projet global, ou à partir de l'application elle même.

C'est une des forces de Django : la construction de vos URLs, i.e. le mapping de votre site, est extrêmement flexible. Vous êtes totalement libre de faire ce que vous voulez (dans les limites de votre nom de domaine bien évidemment).

Nous avons donc vue le principe de base de fonctionnement de Django, mais dites vous bien que c'est une infime (très infime même) partie de toutes les possibilités que peut offrir ce framework !

Il est maintenant temps de rentrer dans le 'dur' de notre projet : la creation d'un backend Django qui communiquera par la suite avec un frontend React via REST Framework.

## Django REST Framework - Brève présentation

Django REST Framework est un ensemble d'outils et de modules permettant de créer de puissantes et flexibles API Web. Il permet la serialisation de données et leur transmission entre Django et React, donc entre le frontend et le backend. Le schéma suivant représente plutôt bien son rôle:

![REST](images_presa/REST.png)

Pour installer ce pack, il faut simplement lancer la commande *pip install djangorestframework*. Déclarons dès maintenant à Django que nous voulons utiliser REST Framework en modifiant comme toujours *settings.py*. La première chose à faire est de déclarer REST Framework dans les applications utilisées:

![settings rest](images_presa/settings_rest.jpg)

Pour l'app corsheader, il faut installer le paquet associé avec pip install django-cors-headers

Comme tout bon add-on qui se respecte, REST Framework peut être parametré sur de multiples aspects. Ces paramètres doivent être déclarés dans *settings.py* via un dictionnaire nommé REST_FRAMEWORK. Voici nos premiers paramètres:

![settings rest params](images_presa/settings_rest_param.jpg)

Le premier paramètre, DEFAULT_PERMISSION_CLASSES, définit la manière dont l'accès aux reqûetes est autorisé ou non. On voit ici qu'aucune restriction n'est imposée, puisque l'on applique AllowAny.

Le second paramètre, DEFAULT_RENDERER_CLASSES, définit la forme sous laquelle les réponses aux requêtes sont transmises. Nous avons pour notre projet imposé que ces réponses soient sous format JSON, nous verrons pourquoi dans la suite.

La liste des différents paramètres et leurs valeurs passables sont donnés dans la documentation de REST Framework, ainsi que tout ce que vous voudrez savoir si vous voulez aller plus loin: https://www.django-rest-framework.org/

## Création de notre modèle Tweet

Nous devons maintenant définir comment sera concrètement défini un tweet dans notre base de données, et dans notre projet. C'est le rôle du modèle *Tweet*. Voyons comment il se construit dans models.py:

![model](images_presa/model.jpg)

Nous construisons un modèle qui est donc une classe qui hérite de models.Model, que nous avons importé. C'est la manière basique de créer un modèle Django.

Nous définissons 3 attributs à notre classe. Ce seront les différents champs de notre base de données. Chaque tweet posté sera donc stocké dans notre base de données où seront renseignés le texte, la date, l'auteur et l'id, un entier attribué automatiquement par Django.

Il existe beaucoup de types de champs de modèles. Cela va du BooleanField au OneToOneField en passant par le ForeignKey. Tous les détails sur ces champs sont explicités dans la doc de Django: https://docs.djangoproject.com/fr/3.2/topics/db/models/

Nous utilisons le champ CharField, qui est le champ correspondant à une chaîne de caractères quelconques.

Le paramètre *max_length* impose une longueur maximale au texte de notre tweet, donc 280 caractères.

Le paramètre *null* précise si un champ vide peut être stocké dans notre base de données avec la valeur NULL. Nous ne voudrons pas de cette propriété dans notre cas.

Le paramètre *blank* précise si un champ vide peut être validé. Cela n'a aucun interêt dans notre cas donc nous lui donnons la valeur *False*.

Le champ DateTimeField a le format d'une date et heure. Nous précisons ici que sa valeur doit être enregistrée automatiquement lors du premier enregistrement du tweet via *auto_now_add*, et qu'il sera ignoré par le processus de validation et l'administration de Django via *editable*.

Le nom de l'auteur sera lui aussi un CharField, mais avec une valeur par défaut. Elle n'est pas nécessaire ici puisque *blank* a la valeur *False*, mais nous l'avons tout de même mis pour détecter d'éventuels bugs.

Enfin nous redéfinissons la méthode spéciale *\__str__* de notre Tweet, pour simplifier son affichage.

Voilà ! Nous avons défini un modèle très simple de tweet, qu'il faut maintenant propager en lançant les commandes:

*python manage.py makemigrations app* puis *python manage.py migrate*

Notre base de données est donc prête à accueillir nos tweets !

## Sérialisation du modèle Tweet

C'est maintenant que REST Framework entre en jeu. Il va permettre de transformer nos objets Tweet, qui sont des classes Python, en données compréhensibles par le frontend React (par exemple du JSON).

L'énorme avantage de cette méthode, c'est qu'elle est totalement réversible. La sérialisation que nous allons définir ici permet également de recevoir des données provenant du frontend (par exemple lorsque quelqu'un poste un tweet) et de les traduire en Tweet tel que défini par Django.

A chaque modèle Django doit correspondre un serialisateur REST Framework. Tous ces serialisateurs sont regroupés dans un programme que nous créeons dans le dossier app et nommons *serializers.py*.

Voyons comment le notre est structuré :

![serializer](images_presa/serializer.jpg)

Cela se présente encore une fois sous forme d'une classe, qui hérite maintenant de serializers.ModelSerializer

Nous précisons notamment le modèle à serialiser et les champs à incorporer dans la serialisation. Dans notre cas tous nos champs seront intéressants à traiter par le frontend. Nous les transmettrons tous.

## Définition de nos vues

Nous avons défini nos objets Tweet, quelle forme ils ont dans la base de données, et leur traduction en JSON. Il reste donc à construire nos vues. Pour notre mini site de tweets, nous allons définir nos vues de manière très simple.

La première sera dédiée à la méthode GET. Si un utilisateur se rend sur notre site, ce dernier va envoyer une requête GET au serveur backend dans le but d'obtenir la liste des tweets. La vue alors utilisée devra renvoyer la liste des tweets sous le format désiré (nombre de tweets à afficher, quelle page, dans quel ordre...).

La seconde sera dédiée à la méthode POST. Si l'utilisateur veut envoyer un tweet, il enverra une requête POST au serveur backend, traduite au travers du serialiseur, et le tweet est crée. Cette creation de tweet doit donc être gérée par une autre vue.

Nous aurions pu évidemment gérer les deux méthodes GET et POST via une même vue, mais par souci de simplicité nous distinguerons les deux.

Voici donc le contenu de *views.py*:

![views](images_presa/views.jpg)

Nous découvrons ici une nouvelle forme de vue, qui se construit comme une classe. Elle hérite de *generics.ListCreateAPIView*, qui permet de représenter très facilement une collection d'objets définis dans un modèle Django. Elle contient:

* La variable *queryset* qui définit quelles données nous devons extraire de notre base de données. Ici nous prenons tous les tweets qu'elle contient.
* La variable *serializer_class* qui définit l'outil de serialisation à utiliser. Ce sera pour nous évidemment TweetSerializer.

Et rien que cette toute petite vue renverra au frontend la liste de tous les tweets enregistrés dans la database, sous forme d'un JSON (puisque c'est le format que nous avons imposé dans *settings.py*).

La deuxième vue est plus classique, puisqu'elle prend la forme d'un fonction, dont l'argument est la requête reçue. Elle crée alors simplement un nouvel objet Tweet, en prenant le texte et l'auteur tels que donnés dans la requête (la date est ajoutée automatiquement comme nous l'avons vu dans la création du modèle). La ligne *tweet.save()* permet ensuite d'enregistrer ce tweet dans la base de données.

Enfin la dernière vue est la vue de test, qui utilise le template index.html dont nous avons parlé précédemment.

## Ajout des routes

Si vous avez bonne mémoire, vous devez vous souvenir que nous avons défini une route pour notre vue *index*, qui était simplement la page d'accueil de notre serveur. Maintenant que nous avons ajouté deux nouvelles vues, il faut définir par quelles routes/quelles URLs nous pourrons y accéder. Il suffit alors de modifier le fichier *app/urls.py*:

![rest urls](images_presa/rest_urls.jpg)

Et le tour est joué ! Vous pouvez maintenant lancer *python manage.py runserver* et vous amuser à tester notre serveur. Allez sur http://localhost:8000/ et créez différents tweets avec différents auteurs. Rendez-vous ensuite sur la page http://localhost:8000/lasttweets et vous verrez la fameuse liste JSON s'afficher.